# 1- Loading libaries and Dataset


In [ ]:
# important library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
from matplotlib.cm import rainbow
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# loading Dataset
df=pd.read_csv("../input/ozone-layer/Ozone.csv")
df.head(25)

In [ ]:
df.columns

# 2- Data Exploration

In [ ]:
df = df.round(1)
df

In [ ]:
# Data validation and preprocessing
df.info()
#Lets now check for null fields
import seaborn as sns
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
df.isnull().sum()

* observation 
    
    NO Null Value Or Missing Data *

In [ ]:
# Duplicates VALUE 
print(f'Duplicates in the dataset: {df.duplicated().sum()}')
print(f'Percentage of duplicates: {df.duplicated().sum()/len(df)*100}%') # if 0.0 % that means No Duplicate data

* obseravation
    
    In datasetb, there are no duplicate values.

In [ ]:
#Cardinality 
df.nunique() # To determine the maximum and minimum number of variations in each column of the dataset

* obseravation
    
    In this dataset Two different classes are present.

# 3- Statistics

In [ ]:
# data describtio
df.describe().T.style.background_gradient()# Significant higher and lower values in a dataset are highlighted

* obseravation
    
    In this dataset some raw showing distict higher values (ex, V70,V67,V62,V57)

# 4- Data Engineering

In [ ]:
# heatmap graph for finding  correlation of column 
print("HEATMAP")
#get the correlation
sns.set_context('poster', font_scale=0.5)
plt.figure(figsize=(80,80))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

* observation

                In this dataset two different group columns interlink.

In [ ]:
#Correlation with output variable
cor_target = abs(cor["V67"])
#Selecting highly correlated features
relevant_features = cor_target #[cor_target>0.5]
relevant_features


In [ ]:
relevant_features = cor_target [cor_target>0.8]
relevant_features

* obseravation
    
    V53,58,62,63,67 group column values  having hi corelation.

In [ ]:
#study the data
sns.set_context('poster', font_scale=0.5)
df.hist(bins=25, grid=False, figsize=(25,18), color='#86bf91', zorder=2, rwidth=0.9)
plt.show()

# 5- Data Visualization

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
#df["y_1"]=df.drop(["Class"],axis=1)

In [ ]:
# px.line(df, x='Class', y= "y_1")

In [ ]:
fig =go.Figure(data=[go.Histogram(x=df, cumulative_enabled=True)])
fig.show()

In [ ]:
#checking the target variable countplot
plt.figure(figsize=(10,10))
sns.countplot(data=df,x = 'Class',palette='plasma')

In [ ]:
sns.set(rc={'figure.figsize':(12,10)})
df['Class'].value_counts().plot.pie(autopct='%1.1f%%', textprops={'fontsize':12}).set_title("class distribution")
df['Class'].value_counts()


In [ ]:
#Box Plotting All features distribution corresponding Target column
i=1
plt.figure(figsize=(60,60))
for c in df.columns[:49]:
    plt.subplot(10,5,i)
    plt.title(f"Boxplot of {c}",fontsize=16)
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    sns.boxplot(y=df[c],x=df['Class'])
    i+=1
plt.show()

# 6- Models on Machina with Baselines

In [ ]:
# Note: Other features will be correlated soon
#Section 2: Baseline Models
#Using a simple model for having a baseline accuracy without removing any features.

# Importing libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Copying dataset for testing baseline
baseline_data = df

# Separating into training and testing set
target = 'Class'
y = baseline_data[target]
X = baseline_data.drop([target], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)


In [ ]:
# Making the model
baseline_model = DecisionTreeClassifier(criterion='entropy', random_state=1)
baseline_model.fit(X_train, y_train)

# Accuracy of the model
baseline_model.score(X_test, y_test)

In [ ]:
# redistribute the unbalanced data with over sampling
from scipy import stats
from imblearn.over_sampling import SMOTE

from sklearn.metrics import confusion_matrix

# display progress of loops
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
df.iloc[:,13:].describe()

In [ ]:
# Normalize the Data
scaler = StandardScaler()
sc_df = scaler.fit_transform(df.iloc[:,:-1])#  target coulumn is microorganism
temp_y = df['Class'].to_numpy()
sc_df = pd.DataFrame(np.hstack((sc_df, temp_y.reshape(-1, 1))), columns=df.columns)
sc_df.head()

In [ ]:
sc_df = sc_df[(np.abs(stats.zscore(df.iloc[:,:-1])) < 3).all(axis=1)]
sc_df.info()


Oversampling with SMOTE (Synthetic Minority Oversampling Technique)

In [ ]:
X = sc_df.drop(['Class'], axis=1).to_numpy()
y = sc_df['Class'].to_numpy()
sm = SMOTE(random_state=42)
X_train_transformed, y_train_transformed = sm.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=42 , shuffle=True)

In [ ]:
# format scoring output
def score_format(model):
    print(f'Accuracy: {round(model * 100, 2)} %')


K-Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print(f'K-Nearest Neighbors')
score_format(knn.score(X_test, y_test))

Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print(f'Random Forest')
score_format(rf.score(X_test, y_test))

Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
print(f'*Gradient Boosting')
score_format(gb.score(X_test, y_test))

In [ ]:
# Multi-Layer Perceptron

In [ ]:
mlp = MLPClassifier(max_iter = 1000)
mlp.fit(X_train, y_train)
score_format(mlp.score(X_test, y_test))


Cross Validation Since we're using SMOTE, we'll be duplicating underrepresented data and this may affect our accuracy Model accuracies suffer from overfitting when models pay too much attention to certain data points during training and cannot predict effectively on unseen points Cross validation will allow us to determine how well each model generalizes on unseen data Cross validation will treat the entire dataset as a training AND test set by splitting into K equal parts, and alternating training/test sets among those K-splits

In [ ]:
models = [knn, rf, gb, mlp]

In [ ]:
for model in models:
    scores = cross_val_score(estimator=model,
                         X=X,
                         y=y,
                         cv=10,
                         n_jobs=-1)
    print(f'CV accuracy: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')


# 7- Hyperparameter Tuning

In [ ]:
# Hyperparameter Tuning
ints_ = [5,7,9, None]
param_grid = [{'n_neighbors': ints_[:-1], 
               'weights': ['uniform', 'distance']
              },
              {'criterion': ['gini', 'entropy'],
               'max_depth': ints_
              },
              {'max_depth': ints_[:-1]},
              {'hidden_layer_sizes': [(100,), (250,)], 
               'activation': ['logistic', 'tanh', 'relu'],
               'max_iter': [10000]
              }]

tuned_models = []
for i in tqdm(range(len(models))):    
    gs = GridSearchCV(models[i], 
                      param_grid[i], 
                      n_jobs = -1,
                      refit = True,
                      cv = 10,
                      return_train_score = True)
    best_model = gs.fit(X, y)
    tuned_models.append(best_model)


# 8- Best outcome using model name

In [ ]:
for model in tuned_models:
    print(f'Best Params: {model.best_estimator_}')
    print(f'Best Score: {model.best_score_:.3f}')
    print()
    print()